
# reddit-wiki-ripper
by Jam#0001

First things first, install the required packages:

In [ ]:
!pip3 install praw regex


Go to the [reddit apps section](https://www.reddit.com/prefs/apps/) and create a new app.

![app creation image](https://i.imgur.com/REpuvlH.png)

Set the following as the client ID and secret

![client ID and secret indicated image](https://i.imgur.com/kqdoRvW.png)

In [ ]:
client_id = input("Client ID")

In [ ]:
client_secret = input("Client Secret")

Then create a read-only instance of PRAW using these details:

In [ ]:
import praw

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent="reddit_wiki_ripper"
)

Now input the subreddit you wish to grab the wiki from

In [ ]:
subreddit = input("Please enter a subreddit")

Last step! Time to grab those wiki pages: 

In [ ]:
import os
import regex

for wikipage in reddit.subreddit(subreddit).wiki:
    # if the subreddit is nested
    if wikipage.name.find('/') != -1:
        dir = os.path.join("./", wikipage.name[:wikipage.name.rindex('/')])
        if not os.path.exists(dir):
            os.makedirs(dir)
    with open("{}.md".format(os.path.join("./", wikipage.name)), "w", encoding="utf-8") as f:
        # fix headings
        formatted_wikipage = regex.sub(r'(?<=\n#?)#(?=[^# ])', '# ', wikipage.content_md)
        formatted_wikipage = formatted_wikipage.replace('https://www.reddit.com/r/{}/wiki/'.format(subreddit), './')
        # replace links so they are to localhost rather than to reddit

        f.write(formatted_wikipage)